# evaluation graph reg 2

the point of this is to evaluate the CNN graph reg model in pytorch, we have it saved, can simply load it in! No decoder or anything, so simply define the model and load it? 

In [2]:
import sys
sys.path.append('/data/leslie/sarthak/caduceus/')
from caduceus.configuration_caduceus import CaduceusConfig
import torch
import numpy as np
import src.dataloaders.datasets.graphreg_dataset as d
import yaml
from omegaconf import OmegaConf
import os
from tqdm import tqdm

ckpt_path = '/data1/lesliec/sarthak/caduceus/outputs/2024-11-27/15-08-13-584820/checkpoints/last.ckpt'

model_cfg_path = os.path.join(os.path.dirname(os.path.dirname(ckpt_path)), '.hydra', 'config.yaml')
cfg = yaml.load(open(model_cfg_path, 'r'), Loader=yaml.FullLoader)

cfg

base_path: /data1/lesliec/sarthak/


{'train': {'seed': 2222,
  'interval': 'epoch',
  'monitor': 'val/loss',
  'mode': 'min',
  'ema': 0.0,
  'test': True,
  'debug': False,
  'ignore_warnings': False,
  'optimizer_param_grouping': {'bias_weight_decay': False,
   'normalization_weight_decay': False},
  'state': {'mode': None, 'n_context': 0, 'n_context_eval': '${.n_context}'},
  'ckpt': 'checkpoints/last.ckpt',
  'disable_dataset': False,
  'validate_at_start': False,
  'pretrained_model_path': None,
  'pretrained_model_strict_load': True,
  'pretrained_model_state_hook': {'_name_': None},
  'post_init_hook': {'_name_': None},
  'layer_decay': {'_name_': None, 'decay': 0.7},
  'gpu_mem': '${eval:"round(float(__import__(\'subprocess\').check_output(\'nvidia-smi -i 0 --query-gpu=memory.total --format=csv,noheader,nounits\', shell=True).strip().decode()) / 1000)"}',
  'global_batch_size': '${dataset.batch_size}'},
 'wandb': {'project': 'dna',
  'group': 'graphreg',
  'job_type': 'training',
  'mode': 'online',
  'name': 'GR

In [3]:
state_dict = torch.load(ckpt_path, map_location='cpu')
dataset_args = cfg['dataset']
split = 'test'
dataset = d.GraphRegDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args.get('kmer_len', None),
                                                            remove_repeats=dataset_args.get('remove_repeats', False),
                                                            has_TSS=dataset_args.get('has_TSS', False),
                                                            clean_data=dataset_args.get('clean_data', True),
                                                            vocab_size=cfg['model']['config']['vocab_size'],
                                                            one_hot = dataset_args['one_hot'],)

In [4]:
dataset_args

{'_name_': 'GraphRegLoader',
 'bed_file': None,
 'fasta_file': None,
 'dataset_name': 'GraphRegLoader',
 'tokenizer_name': 'char',
 'cache_dir': None,
 'max_length': 100000,
 'add_eos': False,
 'batch_size': 128,
 'batch_size_eval': 128,
 'num_workers': 4,
 'shuffle': True,
 'pin_memory': True,
 '__train_len': '${div_up:1_000_000_000, ${.max_length}}',
 '__l_max': '${.max_length}',
 'max_length_val': '${dataset.max_length}',
 'max_length_test': '${dataset.max_length}',
 'pad_max_length': None,
 'rc_aug': False,
 'use_fixed_len_val': False,
 'replace_N_token': False,
 'pad_interval': False,
 'kmer_len': None,
 'cell_type': 'K562',
 'vocab_size': '${model.config.vocab_size}',
 'one_hot': True,
 'clean_data': True,
 'remove_repeats': True}

In [5]:
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
    state_dict["state_dict"], "model."
)
model_state_dict = state_dict["state_dict"]
# need to remove torchmetrics. to remove keys, need to convert to list first
for key in list(model_state_dict.keys()):
    if "torchmetrics" in key:
        model_state_dict.pop(key)


In [6]:
cfg['model'].pop('config')
cfg['model']

{'_name_': 'graph_reg_conv',
 'one_hot': True,
 'input_len': '${dataset.__l_max}'}

In [7]:
from src.models.sequence.conv import GraphRegConvNet

In [8]:
model = GraphRegConvNet(**cfg['model'])
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [9]:
model

GraphRegConvNet(
  (conv1): Conv1d(4, 256, kernel_size=(21,), stride=(1,), padding=same)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dp1): Dropout(p=0.5, inplace=False)
  (conv2): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dp2): Dropout(p=0.5, inplace=False)
  (conv3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp3): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (dp3): Dropout(p=0.5, inplace=False)
  (conv4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, af

In [10]:
a, b = dataset[0]
a.shape, b.shape

(torch.Size([4, 100000]), torch.Size([1000, 3]))

In [11]:
model(a.unsqueeze(0)).shape

torch.Size([1, 1000, 3])

In [12]:
len(dataset)

2756

In [14]:
split = 'train'
dataset = d.GraphRegDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args.get('kmer_len', None),
                                                            remove_repeats=dataset_args.get('remove_repeats', False),
                                                            has_TSS=dataset_args.get('has_TSS', False),
                                                            clean_data=dataset_args.get('clean_data', True),
                                                            one_hot = dataset_args['one_hot'],)

In [15]:
len(dataset)

21165

In [18]:
model = model.eval().cuda()

In [19]:
from tqdm import tqdm
true_vals = np.zeros((len(dataset),1000,3))
pred_vals = np.zeros((len(dataset),1000,3))
#now make a dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
idx = 0
#can also do torch no grad if want bigger batch size, but this is fast enough
for i, (x, y) in enumerate(tqdm(dataloader)):
    pred = model(x.cuda())
    true_vals[idx:idx+x.shape[0]] = y.detach().cpu().numpy()
    pred_vals[idx:idx+x.shape[0]] = pred.detach().cpu().numpy()
    idx += x.shape[0]

100%|██████████| 662/662 [01:32<00:00,  7.15it/s]


In [20]:
print((true_vals.sum(0) == 0).sum()) #now many rows are 0
print((true_vals.sum(1) == 0).sum())
#and for pred
print((pred_vals.sum(0) == 0).sum()) #now many rows are 0
print((pred_vals.sum(1) == 0).sum())

0
3
0
0


In [22]:
#true vals differs a bit, uh oh
true = np.load('/data1/lesliec/sarthak/data/GraphReg/model_out/true_train.npy')
np.allclose(true_vals, true)

False

In [23]:
print(true.shape, true_vals.shape)

(21165, 1000, 3) (21165, 1000, 3)


In [28]:
(true_vals[0] == dataset[0][1].numpy()).all()

True

In [29]:
for i in range(len(dataset)):
    assert (true_vals[i] == dataset[i][1].numpy()).all()

#ok so this is indeed correct, telling us there's some difference between the datasets...

In [31]:
(true-true_vals).max() #yeah that's huge!!

120.16015625

In [32]:
dataset = d.GraphRegDataset('train', 100_000, clean_data=True, one_hot=True, remove_repeats=True)
#so we seem to have given some values that are different?
for i in range(len(dataset)):
    assert (true_vals[i] == dataset[i][1].numpy()).all()

AssertionError: 

In [36]:
a = (true_vals == true).all(2).all(1)
print(a.shape)
print(sum(a)) #so none of these values are corect whereas it seems like above the issue is more some random values

(21165,)
0


In [40]:
i #wait no it's at i = 0. So some issue with the parameters of my dataset class!!
'''dataset = d.GraphRegDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args.get('kmer_len', None),
                                                            remove_repeats=dataset_args.get('remove_repeats', False),
                                                            has_TSS=dataset_args.get('has_TSS', False),
                                                            clean_data=dataset_args.get('clean_data', True),
                                                            vocab_size=cfg['model']['config']['vocab_size'],
                                                            one_hot = dataset_args['one_hot'],)'''

print(dataset_args['rc_aug'], dataset_args['cell_type'], dataset_args['kmer_len'], dataset_args['remove_repeats'], dataset_args['clean_data'], dataset_args['one_hot'])

False K562 None True True True


In [ ]:
#oh I found the main issue, lmfaooo!!! The issue is that we had our original dataset with k562 data, but when we don't specify K562 it defaults to GM12878, so we need to specify K562 in the dataset_args
#ok let's rewrite this as train data for GM12878, remove preds and repredict

In [41]:
true_vals.shape

(21165, 1000, 3)

In [45]:
dataset = d.GraphRegDataset('train', 100_000, clean_data=True, one_hot=True, remove_repeats=True) #this is GM12878
dataset.labels.shape

(21165, 1000, 3)

In [46]:
np.allclose(dataset.labels, true)

True

In [47]:
pred_vals.shape #prediction was indeed k562 so save it out

(21165, 1000, 3)

In [48]:
np.save('/data1/lesliec/sarthak/data/GraphReg/model_out/pred_train_pt_K562.npy', pred_vals)

In [49]:
#and save out the k562 true data
np.save('/data1/lesliec/sarthak/data/GraphReg/model_out/true_train_K562.npy', true_vals)

In [1]:
#let's redo the dataset and evaluate it on the test set

import sys
sys.path.append('/data/leslie/sarthak/caduceus/')
import torch
import numpy as np
import src.dataloaders.datasets.graphreg_dataset as d
import yaml
import os
from tqdm import tqdm

ckpt_path = '/data1/lesliec/sarthak/caduceus/outputs/2024-11-27/15-08-13-584820/checkpoints/last.ckpt'

model_cfg_path = os.path.join(os.path.dirname(os.path.dirname(ckpt_path)), '.hydra', 'config.yaml')
cfg = yaml.load(open(model_cfg_path, 'r'), Loader=yaml.FullLoader)


state_dict = torch.load(ckpt_path, map_location='cpu')
dataset_args = cfg['dataset']
split = 'test'
dataset = d.GraphRegDataset(split, dataset_args['max_length'], rc_aug = dataset_args['rc_aug'],
                                                            cell_type=dataset_args.get('cell_type', None),
                                                            kmer_len=dataset_args.get('kmer_len', None),
                                                            remove_repeats=dataset_args.get('remove_repeats', False),
                                                            has_TSS=dataset_args.get('has_TSS', False),
                                                            clean_data=dataset_args.get('clean_data', True),
                                                            vocab_size=cfg['model']['config']['vocab_size'],
                                                            one_hot = dataset_args['one_hot'],)
dataset_args
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
    state_dict["state_dict"], "model."
)
model_state_dict = state_dict["state_dict"]
# need to remove torchmetrics. to remove keys, need to convert to list first
for key in list(model_state_dict.keys()):
    if "torchmetrics" in key:
        model_state_dict.pop(key)

cfg['model'].pop('config')
cfg['model']
from src.models.sequence.conv import GraphRegConvNet
model = GraphRegConvNet(**cfg['model'])
model.load_state_dict(model_state_dict)

base_path: /data1/lesliec/sarthak/


<All keys matched successfully>

In [2]:
model = model.eval().cuda()
from tqdm import tqdm
true_vals = np.zeros((len(dataset),1000,3))
pred_vals = np.zeros((len(dataset),1000,3))
#now make a dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
idx = 0
#can also do torch no grad if want bigger batch size, but this is fast enough
with torch.no_grad():
    for i, (x, y) in enumerate(tqdm(dataloader)):
        pred = model(x.cuda())
        true_vals[idx:idx+x.shape[0]] = y.detach().cpu().numpy()
        pred_vals[idx:idx+x.shape[0]] = pred.detach().cpu().numpy()
        idx += x.shape[0]

100%|██████████| 87/87 [00:16<00:00,  5.36it/s]


In [3]:
#now load in the true values
true = np.load('/data1/lesliec/sarthak/data/GraphReg/model_out/true_test_K562.npy')

In [4]:
np.allclose(true_vals, true)

True

In [5]:
#now save out the predictions
np.save('/data1/lesliec/sarthak/data/GraphReg/model_out/pred_test_pt_K562.npy', pred_vals)